In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date
import collections
import datetime
import os
import xarray as xr
from cycler import cycler
import matplotlib.patches as mpatches

In [7]:
scenarios = ['ssp126','ssp245','ssp370','ssp585']

modelnames = ['BCC-CSM2-MR', 'CESM2', 'CESM2-WACCM', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 
                  'INM-CM4-8', 'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM']

basins = {'RHINE':'6242', 'RHONE':'6243','PO':'6241', 'DANUBE':'6202', 'TITICACA':'3912', 'SANTA':'3425', 
            'OCONA':'3418', 'MAJES':'3416', 'MAGDALENA':'3227', 'AMAZON':'3203', 'YELCHO':'3429', 
            'VALDIVIA':'3428', 'SERRANO':'3426', 'RAPEL':'3423', 'PUELO':'3422', 'PASCUA':'3420', 
            'PALENA':'3419', 'HUASCO':'3412', 'COPIAPO':'3409', 'CISNES':'3408', 'BIOBIO':'3405', 'BAKER':'3404',
            'AZOPARDO':'3403', 'AISEN':'3401', 'SANTA CRUZ':'3244', 'NEGRO':'3232', 'COLORADO':'3212', 
            'CHICO':'3209', 'TORNEALVEN':'6255', 'THJORSA':'6254', 'OLFUSA':'6237', 'LULEALVEN':'6227', 
            'KUBAN':'6223', 'KALIXALVEN':'6219', 'GLOMAA':'6213', 'DRAMSELVA':'6209', 'SVARTA':'6110', 
            'LAGARFLJOT':'6104', 'JOKULSA A FJOLLUM':'6101', 'CLUTHA':'5406', 'YUKON':'4435', 'TAKU':'4431', 
             'SUSITNA':'4430','STIKINE':'4428', 'SKEENA':'4427','SKAGIT':'4426','NUSHAGAK':'4418','NASS':'4416',
            'KUSKOKWIM':'4414','FRASER':'4410', 'COPPER':'4408', 'COLUMBIA':'4406', 'ALSEK':'4401', 'NELSON':'4125', 
              'MACKENZIE':'4123','COLVILLE':'4110', 'YSYK-KOL':'2919', 'UVS NUUR':'2918', 'TARIM HE':'2914', 
              'TALAS':'2913', 'LAKE BALKHASH':'2910','HAR US NUUR':'2909', 'CHUY':'2905', 'ARAL SEA':'2902', 
              'YELLOW RIVER':'2434', 'MEKONG':'2421', 'KAMCHATKA':'2413', 'SALWEEN':'2319', 'IRRAWADDY':'2310', 
              'INDUS':'2309', 'GANGES':'2306','BRAHMAPUTRA':'2302', 'OB':'2108', 'INDIGIRKA':'2103', 'YANGTZE' : '2433'}

regions = ['RGI 01', 'RGI 02', 'RGI 06', 'RGI 08', 'RGI 10', 'RGI 11', 'RGI 12', 
           'RGI 13', 'RGI 15', 'RGI 16', 'RGI 17', 'RGI 18']

gmodels = ['GloGEM', 'OGGM', 'PyGEM']

fpath0 = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/'

In [8]:
# Create new index using pandas date_range function
start_date = datetime.date(2000, 1, 1)
end_date = datetime.date(2100, 12, 1)
new_indices = pd.date_range(start_date, end_date, freq='MS').strftime('%Y-%m').tolist()

In [9]:
#Importing runoff data
all_rf_data = {}
all_rf_data_annual = {}

for r, region in enumerate(regions):
    fpath = fpath0 + 'Runoff/' + region + '/'
    
    for s, SSP in enumerate(scenarios):
        if SSP not in all_rf_data:
            all_rf_data[SSP] = {}
            all_rf_data_annual[SSP] = {}
        
        for b, basin in enumerate(basins):
            if basin not in all_rf_data[SSP]:
                all_rf_data[SSP][basin] = {}
                all_rf_data_annual[SSP][basin] = {}
            
            for m, GCM in enumerate(modelnames):
                fname = f"runoff_{GCM}_{SSP}_{basin}.csv"
                file_path = os.path.join(fpath, fname)  # Construct the full file path

                if os.path.exists(file_path):
                    try:
                        temp_df = pd.read_csv(file_path, index_col=0)
                        temp_df.index = new_indices
                        temp_df.index = pd.to_datetime(new_indices)
                        
                        all_rf_data[SSP][basin][GCM] = temp_df
                        all_rf_data_annual[SSP][basin][GCM] = temp_df.resample('A').sum()
                    except FileNotFoundError:
                        continue

In [10]:
#Calculating our RMQ

all_rf_w_mean = {}
RMQ_annual = {}

for s, SSP in enumerate(scenarios):
    all_rf_w_mean[SSP] = {}
    RMQ_annual[SSP] = {}
    for b, basin in enumerate(basins):
        all_rf_w_mean[SSP][basin] = {}
        RMQ_annual[SSP][basin] = {}
        for m, GCM in enumerate(modelnames):
            original_df = all_rf_data_annual[SSP][basin][GCM]
            
            # Calculate the mean for each year across the models
            mean_df = original_df.mean(axis=1)
            mean_df = pd.DataFrame({'Mean': mean_df})

            #Creating a new concatenated df
            all_rf_w_mean[SSP][basin][GCM] = pd.concat([original_df, mean_df], axis = 1) 
            
            RMQ_df = original_df.divide(mean_df['Mean'], axis=0)
            RMQ_annual[SSP][basin][GCM] = RMQ_df

In [13]:
basin_names = []
for basin, ID in basins.items():
    basin_names.append(basin)

RMQ = {}
for s, SSP in enumerate(scenarios):
    RMQ[SSP] = {}
    for m, GCM in enumerate(modelnames):
        RMQ_list = []  # List to store averages for each basin
        for b, basin in enumerate(basins):
            # Get the RMQ DataFrame for the current SSP, GCM, and basin
            rmq_df = RMQ_annual[SSP][basin][GCM]
            

            # Calculate the average of annual values for the current basin
            basin_average = rmq_df.mean()
            RMQ_list.append(basin_average)
            
        
        # Store the average values for the current SSP and GCM combination
        RMQ[SSP][GCM] = pd.DataFrame(RMQ_list)
        RMQ[SSP][GCM].index = basin_names

In [27]:
out_df = {}
for s, SSP in enumerate(scenarios):
    out_df[SSP] = {}
    for m, GCM in enumerate(modelnames):
        glo_values = RMQ[SSP][GCM]['GloGEM']
        oggm_values = RMQ[SSP][GCM]['OGGM']
        pygem_values = RMQ[SSP][GCM]['PyGEM']
        

        # Create a dictionary with values for the current basin
        data = {
            'GloGEM': glo_values,
            'OGGM': oggm_values,
            'PyGEM': pygem_values,
        }
                    
        # Create a DataFrame from the list of basin data
        out_df[SSP][GCM] = pd.DataFrame(data)  # Set the index as 'Basin'

In [32]:
out_df[SSP]['CESM2'].loc['RHONE']

GloGEM    1.057973
OGGM      1.123690
PyGEM     0.818337
Name: RHONE, dtype: float64

In [31]:
out_df[SSP]['BCC-CSM2-MR'].loc['RHONE']

GloGEM    1.059942
OGGM      1.116526
PyGEM     0.823531
Name: RHONE, dtype: float64

In [30]:
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/RMQ/'

for SSP in out_df:
    for GCM in out_df[SSP]:
        fname = f"RMQ{GCM}_{SSP}.csv"

        # Define the full path of the output file
        output_path = os.path.join(output_dir, fname)

        # Save the DataFrame as CSV
        out_df[SSP][GCM].to_csv(output_path, header=True, index=True)